In [1]:
import numpy as np

import sensors
import em_stoch

In [2]:
DIST_RATIO = 0.5

In [3]:
Num_sensors1 = 20
Num_emitters1 = 1
sample_size1 = 80
failing_sensors1 = np.array([2, 4], dtype=np.int16)
gap_ratio1 = np.array([0.5, 0.5], dtype=np.float32)
theta1_rad = np.array([0.7]) # Угловые координаты источников (DoA) в радианах
theta1_deg = np.rad2deg(theta1_rad) # Угловые координаты источников (DoA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация сигналов
Q_1 = 2.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шумов
A1 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors1).reshape(-1,1) * 
             np.sin(theta1_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S1 = sensors.gss(Num_emitters1, sample_size1, P_1)
N1 = sensors.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + N1.T).T

Num_sensors2 = 20
Num_emitters2 = 2
sample_size2 = 80
failing_sensors2 = np.array([2, 4], dtype=np.int16)
gap_ratio2 = np.array([0.5, 0.5], dtype=np.float32)
theta2_rad = np.array([-0.5, 0.7], dtype=np.float64) # Угловые координаты источников (DoA) в радианах
theta2_deg = np.rad2deg(theta2_rad) # Угловые координаты источников (DoA) в градусах
P_2 = 1 * np.eye(Num_emitters2, dtype=np.float64) # Ковариация сигналов
Q_2 = 1.1 * np.eye(Num_sensors2, dtype=np.float64) # Ковариация шумов
A2 = (np.exp(-2j * np.pi * DIST_RATIO * np.arange(Num_sensors2).reshape(-1,1) * 
             np.sin(theta2_rad))) # Матрица векторов направленности
# Генерация сигналов, шумов и наблюдений
S2 = sensors.gss(Num_emitters2, sample_size2, P_2)
N2 = sensors.gss(Num_sensors2, sample_size2, Q_2)
X2 = (A2 @ S2.T + N2.T).T

In [4]:
X1_with_mv = sensors.MCAR(X1, failing_sensors1, gap_ratio1)
X2_with_mv = sensors.MCAR(X2, failing_sensors2, gap_ratio2)

In [5]:
X1_with_mv

array([[ 1.47375889-1.19127215j,  0.03175204-1.88842948j,
                nan+0.j        , ..., -0.76397286-2.94873339j,
         0.72662444+0.27358696j,  1.62576741-0.61924096j],
       [-0.75093165-1.09293498j, -0.99980865-0.34039036j,
         2.94445197-1.56076135j, ...,  0.33293359+0.82505105j,
         0.07846728+0.13232699j, -1.07843768+0.39630101j],
       [ 0.76613778+1.51307616j,  0.98300291+0.8604167j ,
        -2.77695094-1.28910182j, ..., -0.60766186+0.05449384j,
        -1.0589133 -1.09211055j, -0.1291021 +1.6502334j ],
       ...,
       [ 1.16245983+2.06921586j,  2.97479457-1.66960229j,
        -2.74510029-0.87284081j, ...,  1.19492776-0.80840476j,
        -2.23063739-0.51135893j,  0.20866592+3.56230975j],
       [ 1.57700122+0.90672591j, -2.1236758 -2.06001117j,
        -1.06651492+2.87218098j, ..., -1.12889979-0.66445606j,
        -0.37962652-0.26983634j,  1.95978744-1.35519038j],
       [-0.56114001+1.07715026j, -0.59027711+0.12132465j,
                nan+0.j       

In [6]:
print(np.trace(sensors.initial_Cov(X1_with_mv)))
print(np.trace(sensors.complex_cov(X1)))

(73.88343486532314+0j)
(63.702954966635296+0j)


In [7]:
EM_theta1_rad, EM_P1, lhd_1 = em_stoch.multi_start_EM(X1_with_mv, 
                                                      Num_emitters1, 
                                                      Q=Q_1, 
                                                      num_of_starts=10, 
                                                      max_iter=100, 
                                                      rtol=1e-6)

0-th start
theta=[0.15335215],P=[[0.00022361+0.j]]
new_angles=[0.14697174]
new_P:
[[0.0013831+0.j]]
likelihood is -3420.7849499048107 on iteration 0
new_angles=[0.14516127]
new_P:
[[0.00239791+0.j]]
likelihood is -3420.735000849567 on iteration 1
new_angles=[0.14464915]
new_P:
[[0.00344439+0.j]]
likelihood is -3420.6933324005813 on iteration 2
new_angles=[0.14333548]
new_P:
[[0.00454323+0.j]]
likelihood is -3420.6459874067295 on iteration 3
new_angles=[0.14194808]
new_P:
[[0.00572074+0.j]]
likelihood is -3420.597196293511 on iteration 4
new_angles=[0.14071442]
new_P:
[[0.00701005+0.j]]
likelihood is -3420.5506992819314 on iteration 5
new_angles=[0.13959401]
new_P:
[[0.00845397+0.j]]
likelihood is -3420.5082213100045 on iteration 6
new_angles=[0.13805368]
new_P:
[[0.01010969+0.j]]
likelihood is -3420.462249086422 on iteration 7
new_angles=[0.13701514]
new_P:
[[0.01205761+0.j]]
likelihood is -3420.435495471408 on iteration 8
new_angles=[0.13615077]
new_P:
[[0.01440659+0.j]]
likelihood is

In [8]:
print(f"EM_theta1_rad={EM_theta1_rad},\nEM_P1={EM_P1}")

EM_theta1_rad=[0.69978431],
EM_P1=[[2.05615129+0.j]]


In [9]:
EM_theta1_deg = np.rad2deg(EM_theta1_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(EM_theta1_deg-theta1_deg)},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta1_rad)-np.sin(theta1_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.01235828],
Погрешность в смысле разности синусов углов:       [0.00016499].


In [10]:
print(np.trace(sensors.initial_Cov(X2_with_mv)))
print(np.trace(sensors.complex_cov(X2)))

(49.65503774285381+0j)
(61.03171543408073+0j)


In [11]:
EM_theta2_rad, EM_P2, lhd_2 = em_stoch.multi_start_EM(X2_with_mv, 
                                                      Num_emitters2, 
                                                      Q=Q_2, 
                                                      num_of_starts=20, 
                                                      max_iter=30,
                                                      rtol=1e-3)

0-th start
theta=[-1.41744418  0.15335215],P=[[0.00276336-1.33017438e-17j 0.        +0.00000000e+00j]
 [0.        +0.00000000e+00j 0.00299515+1.51605636e-17j]]
new_angles=[-0.51963732  0.70927916]
new_P:
[[0.01810106+0.j         0.00048944-0.01147203j]
 [0.00048944+0.01147203j 0.02621943+0.j        ]]
likelihood is -3685.69503259759 on iteration 0
new_angles=[-0.51475708  0.70722992]
new_P:
[[0.12329149+0.j         0.00446904-0.09779392j]
 [0.00446904+0.09779392j 0.1799547 +0.j        ]]
likelihood is -2755.7744668610976 on iteration 1
new_angles=[-0.5107034   0.70523436]
new_P:
[[ 0.79904013+0.j         -0.00614936-0.43461561j]
 [-0.00614936+0.43461561j  0.90560197+0.j        ]]
likelihood is -2175.0857105118434 on iteration 2
new_angles=[-0.50818653  0.70405266]
new_P:
[[ 1.79194503+0.j         -0.07847287-0.28391855j]
 [-0.07847287+0.28391855j  1.49874686+0.j        ]]
likelihood is -2145.9171646346376 on iteration 3
new_angles=[-0.50634917  0.70326718]
new_P:
[[ 2.0167153 +0.j     

In [12]:
print(f"EM_theta2_rad={EM_theta2_rad},\nEM_P2={EM_P2}")

EM_theta2_rad=[-0.50264183  0.69718578],
EM_P2=[[ 1.74325783+0.j         -0.01487166-0.19190085j]
 [-0.01487166+0.19190085j  1.62318067+0.j        ]]


In [13]:
EM_theta2_deg = np.rad2deg(EM_theta2_rad)
print(f"Погрешность в смысле разности углов в градусах: \
      {np.abs(np.sort(EM_theta2_deg)-np.sort(theta2_deg))},")
print(f"Погрешность в смысле разности синусов углов: \
      {np.abs(np.sin(EM_theta2_rad)-np.sin(theta2_rad))}.")

Погрешность в смысле разности углов в градусах:       [0.15136588 0.16124287],
Погрешность в смысле разности синусов углов:       [0.00231675 0.00215498].


In [14]:
em_stoch.incomplete_lkhd(X1_with_mv, theta1_rad, P_1, Q_1)

np.float64(-2824.893641702692)

In [15]:
em_stoch.incomplete_lkhd(X1_with_mv, EM_theta1_rad, EM_P1, Q_1)

np.float64(-2836.4252405078523)

In [16]:
em_stoch.incomplete_lkhd(X2_with_mv, theta2_rad, P_2, Q_2)

np.float64(-2106.416306045104)

In [17]:
em_stoch.incomplete_lkhd(X2_with_mv, EM_theta2_rad, EM_P2, Q_2)

np.float64(-2128.103251844941)